In [1]:
import numpy as np

In [2]:
import os

In [3]:
OUT_DIR = os.path.join("/","data", "km3net", "ml_models_results","cosz")

In [4]:
train_test_dir=os.path.join("/", "data","km3net","Xy_multi_data_files", "train_test_files", "cosz_no_stratify") 

In [5]:
os.makedirs(OUT_DIR, exist_ok=True)

In [6]:
from data_files import get_train_validation_test_files

In [7]:
from tqdm import tqdm

In [8]:
fnames_train, fnames_val, fnames_test, index_filelist = get_train_validation_test_files(n_files=100, 
                                                                                        train_test_dir=train_test_dir)

In [9]:
len(fnames_train)

100

In [10]:
np.load(fnames_train[0])['y']

array([-0.254135,  0.379266,  0.353427, ...,  0.749481,  0.033263,  0.79337 ])

### Training Data

In [11]:
def import_X_y(filelist, set_name, fdata = lambda X: X, ftarget = lambda y: y):
    filename = os.path.join(OUT_DIR, "Xy_{}.npz".format(set_name))
    if os.path.exists(filename):
        with np.load(filename) as xy:
            X = xy["X"]
            y = xy["y"]
    else:
        X = None
        y = None
        for i in tqdm(range(100)):
            with np.load(filelist[i]) as f_t:
                X_f = fdata(f_t['x'])
                y_f = ftarget(f_t['y'])
                if X is None:
                    X = X_f
                    y = y_f
                else:
                    X = np.vstack((X, X_f))
                    y = np.hstack((y, y_f))
    
    np.savez(filename, X=X, y=y)
    return X, y

In [12]:
TZ_AXIS = (2, 3)
def get_Time_Coord(X):
    TZ = np.sum(X, axis=TZ_AXIS)
    TZ = TZ.reshape(TZ.shape[0], TZ.shape[1]*TZ.shape[2])
    return TZ

In [14]:
def to_label(y):
    y[y <= 0] = 0
    y[y > 0] = 1
    return y

In [13]:
X_train, y_train = import_X_y(fnames_train, "train", fdata = get_Time_Coord)

100%|██████████| 100/100 [07:21<00:00,  4.42s/it]


In [15]:
X_train.shape

(165610, 1350)

In [16]:
X_train.size

223573500

In [17]:
y_train.shape

(165610,)

### Test Data

In [19]:
X_test, y_test = import_X_y(fnames_test, "test", fdata = get_Time_Coord)

100%|██████████| 100/100 [02:07<00:00,  1.27s/it]


In [20]:
X_test.shape

(51818, 1350)

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Random Forest

In [24]:
rf_500 = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42, verbose=1)

In [25]:
rf_500.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 97.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 470.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 840.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 901.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=42, verbose=1, warm_start=False)

In [26]:
y_pred_rf_500 = rf_500.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    4.4s finished


In [30]:
acc_rf_500 = accuracy_score(y_test, y_pred_rf_500)

In [31]:
cfm_rf_500 = confusion_matrix(y_test, y_pred_rf_500)

In [32]:
print(cfm_rf_500)

[[20609  2994]
 [ 2729 25486]]


In [33]:
acc_rf_500

0.88955575282720289

In [34]:
np.savetxt(os.path.join(OUT_DIR, "rf_500_test_cls_pred.txt"), y_pred_rf_500)

In [35]:
y_pred_rf500_proba = rf_500.predict_proba(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    4.4s finished


In [36]:
np.savetxt(os.path.join(OUT_DIR, 'rf_500_test_prob_pred.txt'), y_pred_rf500_proba)

In [37]:
from sklearn.externals import joblib

In [38]:
joblib.dump(rf_500, filename=os.path.join(OUT_DIR,'rf_500.pkl'))

['ml_models_results/updown_classification/rf_500.pkl']

In [39]:
rf_100 = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, criterion='entropy', verbose=1)

In [40]:
rf_100.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=1, warm_start=False)

In [41]:
y_pred_rf_100 = rf_100.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.9s finished


In [42]:
acc_rf_100 = accuracy_score(y_test, y_pred_rf_100)
cfm_rf_100 = confusion_matrix(y_test, y_pred_rf_100)

In [43]:
print(acc_rf_100)

0.885599598595


In [44]:
print(cfm_rf_100)

[[20586  3017]
 [ 2911 25304]]


In [45]:
np.savetxt(os.path.join(OUT_DIR, "rf_100_test_cls_pred.txt"), y_pred_rf_100)

In [46]:
y_pred_rf100_proba = rf_100.predict_proba(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.9s finished


In [47]:
np.savetxt(os.path.join(OUT_DIR, 'rf_100_test_prob_pred.txt'), y_pred_rf100_proba)

In [48]:
joblib.dump(rf_100, filename=os.path.join(OUT_DIR,'rf_100.pkl'))

['ml_models_results/updown_classification/rf_100.pkl']

### Adaboost Classifier

In [49]:
bdt = AdaBoostRegressor(random_state=42, n_estimators=500)

In [ ]:
bdt.fit(X_train, y_train)

In [69]:
joblib.dump(bdt, filename=os.path.join(OUT_DIR,'bdt.pkl'))

['./ml_models_results/bdt.pkl']

In [70]:
y_pred_bdt = bdt.predict(X_test)

In [71]:
accuracy_score(y_test, y_pred_bdt)

0.67053919487436797

In [72]:
confusion_matrix(y_test, y_pred_bdt)

array([[10946, 12657],
       [ 4415, 23800]])

In [ ]:
np.savetxt(os.path.join(OUT_DIR, "bdt_test_cls_pred.txt"), y_pred_bdt)